# Deep Painterly Harmonization
### A complete Colab adaptation of the original code including the post-processing stage + optional automatic mask generation

Paper: https://arxiv.org/pdf/1804.03189.pdf

Original repo: https://github.com/luanfujun/deep-painterly-harmonization

Notebook adapted from: https://colab.research.google.com/gist/eyaler/5303782669fb43510d398bd346c6e3e6/deep-painterly-harmonization.ipynb

Notebook link: https://github.com/keerthilogesh/deep-painterly-harmonization-colab/blob/master/demo_of_deep_painterly_harmonization.ipynb

In [ ]:
#@title Setup
#@markdown Takes about 20 minutes...

!apt --purge remove "*cublas*11*" "*cuda*11*"
!apt install cuda-10-0 --reinstall
!rm /usr/local/cuda
!ln -s /usr/local/cuda-10.0 /usr/local/cuda

%cd /content
!git clone --depth 1 https://github.com/nagadomi/distro torch --recursive
!git clone --depth 1 https://github.com/keerthilogesh/deep-painterly-harmonization-colab
%cd /content/torch
!bash install-deps
!yes | ./install.sh
!./install/bin/torch-activate
%cd /content/deep-painterly-harmonization
!sh models/download_models.sh
!wget -nc --no-check-certificate https://raw.githubusercontent.com/Gasp34/PatchMatch/fix_propagation/PatchMatch.py
!make clean && make
import os
if not os.path.exists('data-paper'):
  !mv data data-paper
  !mkdir data 
if not os.path.exists('results-paper'):
  !mv results results-paper
%cd data
!wget -nc --no-check-certificate https://eyalgruss.com/share/rushmore_naive.png
!wget -nc --no-check-certificate https://eyalgruss.com/share/rushmore_target.png
!apt-get install libprotobuf-dev protobuf-compiler
!git config --global url.https://github.com/.insteadOf git://github.com && /content/torch/install/bin/luarocks install loadcaffe

In [ ]:
#@title Harmonize!
#@markdown 1. If you do not provide a mask - will try to generate using missing_mask_tolerance - this works better for original .png files
#@markdown 2. You can set size=0 to use original sizes. If you get a memory error reduce size parameter or set to 700
#@markdown 3. Play with the style_weight parameter. Paper recommends values as: 
#@markdown >* **1** - for Art Nouveau (Modern), Baroque, Early Renaissance, High Renaissance, Mannerism (Late Renaissance), Naive Art (Primitivism), Northern Renaissance, Realism, Surrealism, Symbolism and Ukiyo-e
#@markdown >* **5** - for Abstract Art, Abstract Expressionism, Color Field Painting, Impressionism and Post-Impressionism
#@markdown >* **10** - for Cubism and Expressionism 
#@markdown 4. Intermediate results will be found in: content/deep-painterly-harmonization/results
#@markdown 5. You will get an error if the generated mask is close to uniform, i.e. when the target and naive images are either too similar (e.g. when no object was added) or too different (e.g. when they are .jpg files with unequal original sizes)
#@markdown 6. Please provide the id of the image which needs to be harmonized.

%cd /content/deep-painterly-harmonization
import os
import numpy as np
import cv2
from cv2.ximgproc import createGuidedFilter
from PatchMatch import NNS
from google.colab.patches import cv2_imshow
from google.colab import files
from IPython.utils.capture import capture_output

missing_mask_tolerance =  0#@param {type: "slider", max: 255}
mask_dilate_size = 35 #@param {type: "integer"}
size =  700#@param {type: "integer"}
style_weight =  10#@param {type: "number"}
iterations = 1000 #@param {type: "integer"}
original_colors = False #@param {type: "boolean"}
patch_match_size =  7#@param {type: "integer"}
image_id = 0#@param {type: "integer"}
if not mask_dilate_size % 2:
  mask_dilate_size += 1
if not patch_match_size % 2:
  patch_match_size += 1
original_colors = int(original_colors)
data_folder = 'data-paper'
result_folder = 'results'

def resize(im, size):
  h, w = im.shape[:2]
  if not max(h, w) == size:
    if h > w:
      w = size * w // h
      h = size
    else:
      h = size * h // w
      w = size
  return cv2.resize(im, (w, h))

os.makedirs(result_folder, exist_ok=True)
images = os.listdir(data_folder)
images = [i for i in images if str(image_id) + "_" in i]
for naive in images:
  if '_naive' not in naive:
    continue
  naive = os.path.join(data_folder, naive)
  print("Naive image path: ", naive)
  naive_im = cv2.imread(naive)
  print("Naive image shape: ", naive_im.shape)
  if not size:
    new_size = max(naive_im.shape[:2])
  else:
    new_size = size
  prefix = naive.split('_naive')[0]
  for target in images:
    target = os.path.join(data_folder, target)
    if target.startswith(prefix + '_target'):
      break
  else:
    print('Could not find', prefix + '_target')
    continue
  for mask in images:
    mask = os.path.join(data_folder, mask)
    if mask.startswith(prefix + '_mask'):
      break
  else:
    print('Could not find', prefix + '_mask', '- will create mask!')
    print("Target image path: ", target)
    target_im = cv2.imread(target)
    print("Target image shape: ", target_im.shape)
    naive_im = resize(naive_im, new_size)
    if target_im.shape[:2] != naive_im.shape[:2]:
      target_im = cv2.resize(target_im, naive_im.shape[:2][::-1])
    mask = prefix + '_genmask' + os.path.splitext(naive)[1]
    cv2.imwrite(mask, (np.max(abs(naive_im-target_im), axis=-1) > missing_mask_tolerance)*255)
  print(prefix)
  print("Mask image path: ", mask)
  mask_im = cv2.imread(mask)
  print("Mask image shape: ", mask_im.shape)
  h, w, c = mask_im.shape
  if c == 3:
    mask_im = mask_im[..., 0]  
  mask_im = resize(mask_im, new_size)
  dilated_im = cv2.GaussianBlur(mask_im / 255, (mask_dilate_size, mask_dilate_size), mask_dilate_size / 3)
  dilated_im[dilated_im > 0.1] = 255
  dilated_im[dilated_im <= 0.1] = 0
  dilated = prefix + '_dilated' + os.path.splitext(mask)[1]
  cv2.imwrite(dilated, dilated_im)
  res_prefix = os.path.join(result_folder, prefix.split('/')[-1])
  inter = res_prefix + '_inter_res.jpg'
  final = res_prefix + '_final_res.jpg'
  final1 = res_prefix + '_final_res1.jpg'
  final2 = res_prefix + '_final_res2.jpg'
  with open('style.txt', 'w') as f:
    f.write(f'idx=0, classifed label=sheker kolshehu, weight={style_weight}\n')
  common_args = f'-content_image {naive} -style_image {target} -tmask_image {mask} -mask_image {dilated} -gpu 0 -original_colors {original_colors} -image_size {new_size} -print_iter 100 -save_iter 0'
  print(f"/content/torch/install/bin/th neural_gram.lua {common_args} -output_image {inter} && /content/torch/install/bin/th neural_paint.lua {common_args} -cnnmrf_image {inter} -wikiart_fn style.txt -output_image {final} -num_iterations {iterations}")
  with capture_output() as cap:
    !/content/torch/install/bin/th neural_gram.lua $common_args -output_image $inter && /content/torch/install/bin/th neural_paint.lua $common_args -cnnmrf_image $inter -wikiart_fn style.txt -output_image $final -num_iterations $iterations
    out = cap.stdout
  if 'out of memory' in out:
    print('\nOut of memory error! Reduce size parameter or set to 700')
    break
  
  tr = 3
  dilated_im = cv2.GaussianBlur(mask_im / 255, (tr * 2 + 1, tr * 2 + 1), tr)
  dilated_im[dilated_im > 0.01] = 1
  dilated_im[dilated_im <= 0.01] = 0
  dilated_im = cv2.GaussianBlur(dilated_im, (tr * 2 + 1, tr * 2 + 1), tr)
  
  r = 2  # try 2, 4, 8
  eps = 0.1**2  # try 0.1**2, 0.2**2, 0.4**2
  print("Final image read path: ", final)
  final_im = cv2.imread(final)
  print("Final image shape: ", final_im.shape)
  final_im = cv2.cvtColor(final_im, cv2.COLOR_BGR2LAB)
  guided = createGuidedFilter(naive_im, r, eps * 255 * 255)
  final_im[..., 1] = guided.filter(final_im[..., 1])
  final_im[..., 2] = guided.filter(final_im[..., 2])
  final_im = cv2.cvtColor(final_im, cv2.COLOR_LAB2BGR)
  cv2.imwrite(final1, final_im)

  iter = 5
  target_im = cv2.imread(target)
  if target_im.shape[:2] != final_im.shape[:2]:
    target_im = cv2.resize(target_im, final_im.shape[:2][::-1])
  with capture_output() as cap:
    ann, dist, score_list = NNS(final_im, target_im, patch_match_size, iter)
  final_im2_base = np.zeros_like(final_im)
  for i in range(final_im.shape[0]):
    for j in range(final_im.shape[1]):
      final_im2_base[i, j] = target_im[ann[i, j][0], ann[i, j][1]]
    
  fr = 3
  final_im_base = cv2.GaussianBlur(final_im, (fr * 2 + 1, fr * 2 + 1), fr)
  final_im2 = final_im2_base.astype(np.float32) + final_im - final_im_base
  dilated_im = dilated_im[..., np.newaxis]
  final_im2 = final_im2 * dilated_im + target_im.astype(np.float32) * (1 - dilated_im)
  
  cv2.imwrite(final2, final_im2)
  cv2_imshow(final_im2)
  # files.download(final2)
